## Main File for Project! All important functions are here

#### Jack Cells / Helpers

In [ ]:
%pip install opencv-python
%pip install os
%pip install numpy
%pip install segment_anything
%pip install torch torchvision --extra-index-url https://download.pytorch.org/whl/cu118

import os
import cv2
import numpy as np
from segment_anything import SamPredictor, sam_model_registry, SamAutomaticMaskGenerator

In [ ]:
def get_mask_for_point(image_path, point, model_type="default", checkpoint_path=None):
    try:
        # Load SAM model
        if not os.path.exists(checkpoint_path):
            raise FileNotFoundError(f"Checkpoint file not found at: {checkpoint_path}")
        sam = sam_model_registry[model_type](checkpoint=checkpoint_path)
        predictor = SamPredictor(sam)

        # Load/set img
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        predictor.set_image(image)

        # Generate mask
        input_point = np.array([point])
        input_label = np.array([1])  # For foreground
        masks, _, _ = predictor.predict(
            point_coords=input_point,
            point_labels=input_label,
            multimask_output=False,  # Return only the best mask
        )
        return masks[0]  # Return the first mask (best mask)

    except Exception as e:
        print(f"Error: {e}")
        return None

In [ ]:
# Example usage
image_path = "test.jpg"
checkpoint_path_small = "sam_vit_b_01ec64.pth"
point = (1000, 1000) 

mask = get_mask_for_point(image_path, point, model_type="vit_b", checkpoint_path=checkpoint_path_small)

if mask is not None:
    print("Mask shape:", mask.shape)
    # You can further process or display the mask here.  For example:
    import matplotlib.pyplot as plt
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    plt.figure()
    plt.imshow(image)
    plt.imshow(mask, cmap='gray', alpha=0.5)  # Overlay the mask
    plt.scatter(point[0], point[1], c='red', s=10) # added line
    plt.title('Image with Mask Overlay')
    plt.show()

else: print("Failed to generate mask.")

# main.ipynb

In this notebook, we walk through our methods for detecting the positions of darts on a dartboard, and returning a score based on their positions. We have a few different methods we tried to implement, and experimented with many ideas. 

plan for sections to include underneath this (to show our thought process). We can delete this after we place all our code in

NAIVE IMPLEMENTATION:
- First, some experimentation with the frontal view
  - make it clear that while this is easier for a user, a front view of the darts just does not provide enough information about the locations of the tips. In many cases, you cannot see where the tips hit the board by taking an image from this position
  - show how when doing a difference check between a blank board and one with darts, the changes in orientation cause it to be hard to pinpoint where darts landed, let alone where their tips are
- Show our idea with taking a photo from the right side of the board, and then computing a homography to warp it into a frontal view. It gives us the best of both worlds: the information gained by looking from the side, but also a frontal view that allows us to standardize our scoring (we get the same size photo each time, with the center of the board centered thanks to the ArUco's)
- Maybe briefly mention that we attempted some template matching.. and while it was successful in cropping the board, the issues of scale (sometimes someone would be farther away when they took the picture of the board, sometimes closer) ended up causing issues. Having to search for versions of the template that were smaller took a lot of time compared to using pre-placed ArUco's.
- So, now we are at the point where we have a warped frontal image of the board.. show how we initially tried to detect darts
  - this would be by using a neon yellow mask, yellow darts, and finding countours (taking the farthest right point)
- Show how we intially converted these locations on the image to score (physical COCO Json mapping)

EXTENSIONS:
- how can we find regions of the dartboard without a manually generated mapping? If we want this to be able to work well on any dart board, we cannot assume that ArUco's will get us the same homography every time.. we need something more robust which can look at the image and find regions (enter Rachel's ideas and code + some of reed's)
- How can we detect darts without the need for spray painting them neon yellow? (enter Jonathan's work with ML)